# 🎬 Desafio Cientista de Dados - Análise Cinematográfica PProductions

## 📋 Objetivo
Analisar dados cinematográficos do IMDB para orientar o estúdio PProductions na escolha do próximo filme a ser produzido.

**Deliverables:**
1. Análise exploratória completa (EDA)
2. Modelo preditivo para avaliações IMDB
3. Insights e recomendações estratégicas
4. Predição para 'The Shawshank Redemption'

---

## 📦 1. Configuração e Imports

In [ ]:
# Imports principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import pickle
import warnings
from collections import Counter
from pathlib import Path

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Configurações
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("🎬 Ambiente configurado com sucesso!")

## 📊 2. Carregamento e Preparação dos Dados

In [ ]:
def load_movie_data(data_path='csvjson.json'):
    """Carrega dados do arquivo ou gera simulados"""
    
    if Path(data_path).exists():
        print(f"📁 Carregando dados de {data_path}...")
        with open(data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
    else:
        print("🎲 Arquivo não encontrado. Gerando dados simulados...")
        df = generate_simulated_data()
    
    return df

def generate_simulated_data():
    """Gera dados simulados realistas"""
    np.random.seed(42)
    n_movies = 999
    
    # Dados baseados em padrões reais do IMDB Top 1000
    years = np.random.randint(1920, 2023, n_movies)
    ratings = np.random.normal(8.1, 0.8, n_movies)
    ratings = np.clip(ratings, 6.0, 9.5)
    
    meta_scores = np.random.normal(75, 15, n_movies)
    meta_scores = np.clip(meta_scores, 20, 100)
    
    votes = np.random.lognormal(13, 1, n_movies).astype(int)
    votes = np.clip(votes, 10000, 3000000)
    
    runtimes = np.random.normal(120, 25, n_movies).astype(int)
    runtimes = np.clip(runtimes, 70, 250)
    
    genres = ['Drama', 'Action', 'Comedy', 'Crime', 'Adventure', 'Thriller', 'Romance', 'Sci-Fi', 'Horror', 'Biography']
    certificates = ['U', 'UA', 'A', 'R', 'PG']
    
    df = pd.DataFrame({
        'Rank': range(1, n_movies + 1),
        'Series_Title': [f'Movie_{i}' for i in range(1, n_movies + 1)],
        'Released_Year': years,
        'Certificate': np.random.choice(certificates, n_movies),
        'Runtime': [f'{runtime} min' for runtime in runtimes],
        'Genre': [np.random.choice(genres) + (', ' + np.random.choice(genres) if np.random.random() > 0.6 else '') for _ in range(n_movies)],
        'IMDB_Rating': np.round(ratings, 1),
        'Overview': [f'A compelling story about {np.random.choice(["love", "war", "adventure", "mystery", "family", "friendship"])}...' for _ in range(n_movies)],
        'Meta_score': np.round(meta_scores, 0),
        'Director': [f'Director_{i}' for i in range(1, n_movies + 1)],
        'Star1': [f'Actor_{i}_1' for i in range(1, n_movies + 1)],
        'Star2': [f'Actor_{i}_2' for i in range(1, n_movies + 1)],
        'Star3': [f'Actor_{i}_3' for i in range(1, n_movies + 1)],
        'Star4': [f'Actor_{i}_4' for i in range(1, n_movies + 1)],
        'No_of_Votes': votes,
        'Gross': [f'{gross:,}' for gross in np.random.lognormal(17, 1.5, n_movies).astype(int)]
    })
    
    return df

# Carregar dados
df = load_movie_data()
print(f"✅ Dataset carregado: {df.shape[0]} filmes, {df.shape[1]} colunas")
df.head()

In [ ]:
def clean_data(df):
    """Limpa e prepara os dados"""
    
    # Extrair minutos do runtime
    def extract_minutes(runtime_str):
        if pd.isna(runtime_str):
            return np.nan
        return int(re.findall(r'\d+', str(runtime_str))[0])
    
    # Limpar faturamento
    def clean_gross(gross_str):
        if pd.isna(gross_str) or gross_str == '':
            return np.nan
        return float(str(gross_str).replace(',', ''))
    
    # Aplicar limpezas
    df['Runtime_mins'] = df['Runtime'].apply(extract_minutes)
    df['Gross_numeric'] = df['Gross'].apply(clean_gross)
    df['Meta_score'] = pd.to_numeric(df['Meta_score'], errors='coerce')
    df['Primary_Genre'] = df['Genre'].str.split(',').str[0]
    df['Decade'] = (df['Released_Year'] // 10) * 10
    
    return df

# Limpar dados
df = clean_data(df)
print("🧹 Dados limpos e preparados!")

# Visualizar informações básicas
print(f"\n📊 Informações do Dataset:")
print(f"Período: {df['Released_Year'].min()} - {df['Released_Year'].max()}")
print(f"Ratings: {df['IMDB_Rating'].min():.1f} - {df['IMDB_Rating'].max():.1f}")
print(f"Média de rating: {df['IMDB_Rating'].mean():.2f}")

df.info()

## 📈 3. Análise Exploratória de Dados (EDA)

### 3.1 Estatísticas Descritivas

In [ ]:
# Estatísticas descritivas das variáveis numéricas
numeric_cols = ['IMDB_Rating', 'Meta_score', 'Runtime_mins', 'No_of_Votes', 'Gross_numeric', 'Released_Year']
stats_df = df[numeric_cols].describe()

print("📊 ESTATÍSTICAS DESCRITIVAS")
print("=" * 50)
display(stats_df.round(2))

: 

### 3.2 Distribuição das Avaliações IMDB

In [ ]:
# Criar figura com subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🎬 Análise de Distribuições - Dataset de Filmes', fontsize=16, fontweight='bold')

# 1. Distribuição de Ratings IMDB
axes[0,0].hist(df['IMDB_Rating'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].axvline(df['IMDB_Rating'].mean(), color='red', linestyle='--', label=f'Média: {df["IMDB_Rating"].mean():.2f}')
axes[0,0].set_title('Distribuição das Avaliações IMDB')
axes[0,0].set_xlabel('Rating IMDB')
axes[0,0].set_ylabel('Frequência')
axes[0,0].legend()

# 2. Distribuição de Meta Scores
axes[0,1].hist(df['Meta_score'].dropna(), bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0,1].axvline(df['Meta_score'].mean(), color='red', linestyle='--', label=f'Média: {df["Meta_score"].mean():.1f}')
axes[0,1].set_title('Distribuição de Meta Scores')
axes[0,1].set_xlabel('Meta Score')
axes[0,1].set_ylabel('Frequência')
axes[0,1].legend()

# 3. Distribuição de Runtime
axes[1,0].hist(df['Runtime_mins'], bins=30, alpha=0.7, color='orange', edgecolor='black')
axes[1,0].axvline(df['Runtime_mins'].mean(), color='red', linestyle='--', label=f'Média: {df["Runtime_mins"].mean():.0f} min')
axes[1,0].set_title('Distribuição de Duração dos Filmes')
axes[1,0].set_xlabel('Duração (minutos)')
axes[1,0].set_ylabel('Frequência')
axes[1,0].legend()

# 4. Box plot de Ratings por Década
decade_data = [df[df['Decade'] == decade]['IMDB_Rating'] for decade in sorted(df['Decade'].unique())]
decade_labels = [f"{int(decade)}s" for decade in sorted(df['Decade'].unique())]
axes[1,1].boxplot(decade_data, labels=decade_labels)
axes[1,1].set_title('Ratings por Década')
axes[1,1].set_xlabel('Década')
axes[1,1].set_ylabel('Rating IMDB')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 3.3 Análise Temporal

In [ ]:
# Análise por década
decade_analysis = df.groupby('Decade').agg({
    'IMDB_Rating': ['mean', 'std', 'count'],
    'Gross_numeric': 'mean',
    'Runtime_mins': 'mean',
    'Meta_score': 'mean'
}).round(2)

decade_analysis.columns = ['Rating_Mean', 'Rating_Std', 'Count', 'Gross_Mean', 'Runtime_Mean', 'MetaScore_Mean']

print("📅 ANÁLISE POR DÉCADA")
print("=" * 50)
display(decade_analysis)

# Visualização da evolução temporal
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Rating médio por década
decade_analysis['Rating_Mean'].plot(kind='line', marker='o', ax=axes[0], color='blue', linewidth=2)
axes[0].set_title('📈 Evolução do Rating Médio por Década')
axes[0].set_xlabel('Década')
axes[0].set_ylabel('Rating IMDB Médio')
axes[0].grid(True, alpha=0.3)

# Faturamento médio por década (em milhões)
(decade_analysis['Gross_Mean'] / 1_000_000).plot(kind='bar', ax=axes[1], color='green', alpha=0.7)
axes[1].set_title('💰 Faturamento Médio por Década (Milhões USD)')
axes[1].set_xlabel('Década')
axes[1].set_ylabel('Faturamento Médio (Milhões)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 3.4 Análise de Gêneros

In [ ]:
# Contagem de gêneros (incluindo múltiplos gêneros)
all_genres = []
for genre_str in df['Genre']:
    if pd.notna(genre_str):
        genres = [g.strip() for g in str(genre_str).split(',')]
        all_genres.extend(genres)

genre_counts = Counter(all_genres)

print("🎭 TOP 10 GÊNEROS MAIS COMUNS")
print("=" * 40)
for genre, count in genre_counts.most_common(10):
    print(f"{genre}: {count} filmes")

# Análise por gênero principal
genre_analysis = df.groupby('Primary_Genre').agg({
    'IMDB_Rating': ['mean', 'count', 'std'],
    'Gross_numeric': 'mean',
    'Runtime_mins': 'mean'
}).round(2)

genre_analysis.columns = ['Rating_Mean', 'Count', 'Rating_Std', 'Gross_Mean', 'Runtime_Mean']
genre_analysis = genre_analysis.sort_values('Rating_Mean', ascending=False)

print(f"\n🏆 ANÁLISE POR GÊNERO PRINCIPAL")
print("=" * 50)
display(genre_analysis)

# Visualização dos gêneros
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Top 10 gêneros mais comuns
top_genres = genre_counts.most_common(10)
genres_names = [g[0] for g in top_genres]
genres_counts = [g[1] for g in top_genres]

axes[0,0].barh(genres_names, genres_counts, color='skyblue')
axes[0,0].set_title('🎭 Top 10 Gêneros Mais Comuns')
axes[0,0].set_xlabel('Número de Filmes')

# Rating médio por gênero (top 10)
top_genre_ratings = genre_analysis.head(10)
axes[0,1].bar(range(len(top_genre_ratings)), top_genre_ratings['Rating_Mean'], color='lightgreen')
axes[0,1].set_title('🏆 Rating Médio por Gênero (Top 10)')
axes[0,1].set_xlabel('Gênero')
axes[0,1].set_ylabel('Rating IMDB Médio')
axes[0,1].set_xticks(range(len(top_genre_ratings)))
axes[0,1].set_xticklabels(top_genre_ratings.index, rotation=45, ha='right')

# Scatter plot: Rating vs Faturamento por gênero
for genre in df['Primary_Genre'].unique()[:8]:  # Top 8 gêneros
    genre_data = df[df['Primary_Genre'] == genre]
    axes[1,0].scatter(genre_data['IMDB_Rating'], genre_data['Gross_numeric']/1_000_000, 
                     alpha=0.6, label=genre, s=30)

axes[1,0].set_title('💰 Rating vs Faturamento por Gênero')
axes[1,0].set_xlabel('Rating IMDB')
axes[1,0].set_ylabel('Faturamento (Milhões USD)')
axes[1,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Runtime médio por gênero
runtime_by_genre = df.groupby('Primary_Genre')['Runtime_mins'].mean().sort_values(ascending=False).head(10)
axes[1,1].bar(range(len(runtime_by_genre)), runtime_by_genre.values, color='orange')
axes[1,1].set_title('⏱️ Duração Média por Gênero')
axes[1,1].set_xlabel('Gênero')
axes[1,1].set_ylabel('Duração Média (min)')
axes[1,1].set_xticks(range(len(runtime_by_genre)))
axes[1,1].set_xticklabels(runtime_by_genre.index, rotation=45, ha='right')

plt.tight_layout()
plt.show()

### 3.5 Análise de Correlações

In [ ]:
# Matriz de correlação
correlation_cols = ['IMDB_Rating', 'Meta_score', 'Runtime_mins', 'No_of_Votes', 'Gross_numeric', 'Released_Year']
correlation_matrix = df[correlation_cols].corr()

print("🔗 CORRELAÇÕES COM IMDB RATING")
print("=" * 40)
correlations_with_rating = correlation_matrix['IMDB_Rating'].sort_values(ascending=False)
for var, corr in correlations_with_rating.items():
    if var != 'IMDB_Rating':
        print(f"{var}: {corr:.3f}")

# Heatmap de correlações
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlBu_r', center=0, 
            square=True, fmt='.3f', linewidths=0.5)
plt.title('🔗 Matriz de Correlação - Variáveis Numéricas')
plt.tight_layout()
plt.show()

# Scatter plots das correlações mais interessantes
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# IMDB Rating vs Meta Score
axes[0,0].scatter(df['Meta_score'], df['IMDB_Rating'], alpha=0.6, color='blue')
axes[0,0].set_xlabel('Meta Score')
axes[0,0].set_ylabel('IMDB Rating')
axes[0,0].set_title('📊 IMDB Rating vs Meta Score')

# IMDB Rating vs Number of Votes
axes[0,1].scatter(df['No_of_Votes']/1000, df['IMDB_Rating'], alpha=0.6, color='green')
axes[0,1].set_xlabel('Número de Votos (milhares)')
axes[0,1].set_ylabel('IMDB Rating')
axes[0,1].set_title('🗳️ IMDB Rating vs Número de Votos')

# Runtime vs IMDB Rating
axes[1,0].scatter(df['Runtime_mins'], df['IMDB_Rating'], alpha=0.6, color='orange')
axes[1,0].set_xlabel('Duração (minutos)')
axes[1,0].set_ylabel('IMDB Rating')
axes[1,0].set_title('⏱️ Duração vs IMDB Rating')

# Gross vs IMDB Rating
axes[1,1].scatter(df['Gross_numeric']/1_000_000, df['IMDB_Rating'], alpha=0.6, color='red')
axes[1,1].set_xlabel('Faturamento (Milhões USD)')
axes[1,1].set_ylabel('IMDB Rating')
axes[1,1].set_title('💰 Faturamento vs IMDB Rating')

plt.tight_layout()
plt.show()

### 3.6 Análise de Filmes de Alto Desempenho

In [ ]:
# Definir critérios de alto desempenho
high_rating = df['IMDB_Rating'] >= 8.5
high_votes = df['No_of_Votes'] >= df['No_of_Votes'].quantile(0.75)
high_gross = df['Gross_numeric'] >= df['Gross_numeric'].quantile(0.8)

print("🏆 ANÁLISE DE ALTO DESEMPENHO")
print("=" * 40)
print(f"Filmes com alta avaliação (≥8.5): {high_rating.sum()}")
print(f"Filmes com muitos votos (top 25%): {high_votes.sum()}")
print(f"Filmes com alto faturamento (top 20%): {high_gross.sum()}")

# Filmes que atendem a todos os critérios
triple_success = high_rating & high_votes & high_gross
print(f"Filmes com excelência em TODOS os critérios: {triple_success.sum()}")

if triple_success.sum() > 0:
    success_movies = df[triple_success]
    print(f"\n🌟 CARACTERÍSTICAS DOS FILMES DE MAIOR SUCESSO:")
    print(f"Rating médio: {success_movies['IMDB_Rating'].mean():.2f}")
    print(f"Meta score médio: {success_movies['Meta_score'].mean():.1f}")
    print(f"Runtime médio: {success_movies['Runtime_mins'].mean():.0f} minutos")
    print(f"Faturamento médio: ${success_movies['Gross_numeric'].mean()/1_000_000:.0f}M")
    
    # Gêneros mais comuns nos filmes de sucesso
    success_genres = []
    for genre_str in success_movies['Genre']:
        if pd.notna(genre_str):
            genres = [g.strip() for g in str(genre_str).split(',')]
            success_genres.extend(genres)
    
    success_genre_counts = Counter(success_genres)
    print(f"\nGêneros mais comuns nos filmes de sucesso:")
    for genre, count in success_genre_counts.most_common(5):
        print(f"  {genre}: {count}")

# Análise de filmes lucrativos vs artísticos
artistic_films = df[(df['Meta_score'] >= 85) & (df['Gross_numeric'] <= df['Gross_numeric'].median())]
blockbusters = df[(df['Gross_numeric'] >= df['Gross_numeric'].quantile(0.9)) & (df['Meta_score'] <= 70)]

print(f"\n🎨 ANÁLISE: ARTE vs COMERCIAL")
print(f"Filmes 'artísticos' (alta crítica, baixo faturamento): {len(artistic_films)}")
if len(artistic_films) > 0:
    print(f"  Rating médio: {artistic_films['IMDB_Rating'].mean():.2f}")

print(f"Blockbusters (alto faturamento, crítica moderada): {len(blockbusters)}")
if len(blockbusters) > 0:
    print(f"  Rating médio: {blockbusters['IMDB_Rating'].mean():.2f}")

## 🤖 4. Modelagem Preditiva

### 4.1 Preparação das Features

In [ ]:
print("🔧 PREPARANDO FEATURES PARA O MODELO")
print("=" * 40)

# Features numéricas
numeric_features = ['Meta_score', 'Runtime_mins', 'No_of_Votes', 'Gross_numeric', 'Released_Year']

# Encoding de features categóricas
le_cert = LabelEncoder()
le_genre = LabelEncoder()

df['Certificate_encoded'] = le_cert.fit_transform(df['Certificate'])
df['Primary_Genre_encoded'] = le_genre.fit_transform(df['Primary_Genre'])

# Features finais
features = numeric_features + ['Certificate_encoded', 'Primary_Genre_encoded']
X = df[features].fillna(df[features].median())
y = df['IMDB_Rating']

print(f"Features utilizadas: {features}")
print(f"Shape dos dados: X={X.shape}, y={y.shape}")

# Verificar dados
display(X.head())

### 4.2 Treinamento e Avaliação de Modelos

In [ ]:
print("🏋️‍♂️ TREINANDO MODELOS")
print("=" * 30)

# Split dos dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelos para testar
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

model_results = {}
best_score = -np.inf
best_model_name = None
best_model = None

for name, model in models.items():
    print(f"\n🔄 Testando {name}...")
    
    # Treinar modelo
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calcular métricas
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    model_results[name] = {
        'model': model,
        'r2': r2,
        'mae': mae,
        'rmse': rmse,
        'predictions': y_pred
    }
    
    print(f"   R² Score: {r2:.4f}")
    print(f"   MAE: {mae:.4f}")
    print(f"   RMSE: {rmse:.4f}")
    
    # Atualizar melhor modelo
    if r2 > best_score:
        best_score = r2
        best_model_name = name
        best_model = model

print(f"\n🏆 MELHOR MODELO: {best_model_name}")
print(f"🎯 R² Score: {best_score:.4f}")
print(f"🎯 MAE: {model_results[best_model_name]['mae']:.4f}")

# Salvar informações importantes
use_scaling = (best_model_name == 'Linear Regression')

# Feature importance (se Random Forest)
if best_model_name == 'Random Forest':
    feature_importance = pd.DataFrame({
        'feature': features,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n📊 IMPORTÂNCIA DAS FEATURES:")
    display(feature_importance)

### 4.3 Visualização dos Resultados do Modelo

In [ ]:
# Visualizar resultados
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle(f'🤖 Avaliação do Modelo: {best_model_name}', fontsize=16, fontweight='bold')

# 1. Predições vs Valores Reais
y_pred_best = model_results[best_model_name]['predictions']
axes[0,0].scatter(y_test, y_pred_best, alpha=0.6, color='blue')
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,0].set_xlabel('Valores Reais')
axes[0,0].set_ylabel('Predições')
axes[0,0].set_title('Predições vs Valores Reais')

# 2. Resíduos
residuals = y_test - y_pred_best
axes[0,1].scatter(y_pred_best, residuals, alpha=0.6, color='green')
axes[0,1].axhline(y=0, color='red', linestyle='--')
axes[0,1].set_xlabel('Predições')
axes[0,1].set_ylabel('Resíduos')
axes[0,1].set_title('Análise de Resíduos')

# 3. Distribuição dos resíduos
axes[1,0].hist(residuals, bins=20, alpha=0.7, color='purple', edgecolor='black')
axes[1,0].set_xlabel('Resíduos')
axes[1,0].set_ylabel('Frequência')
axes[1,0].set_title('Distribuição dos Resíduos')

# 4. Feature importance (se Random Forest)
if best_model_name == 'Random Forest':
    feature_importance.plot(x='feature', y='importance', kind='barh', ax=axes[1,1], color='orange')
    axes[1,1].set_title('Importância das Features')
    axes[1,1].set_xlabel('Importância')
else:
    # Gráfico de comparação de modelos
    model_names = list(model_results.keys())
    r2_scores = [model_results[name]['r2'] for name in model_names]
    
    axes[1,1].bar(model_names, r2_scores, color=['skyblue', 'lightgreen'])
    axes[1,1].set_title('Comparação de Modelos (R²)')
    axes[1,1].set_ylabel('R² Score')

plt.tight_layout()
plt.show()

# Métricas finais
print("📈 MÉTRICAS FINAIS DO MODELO")
print("=" * 35)
for name, results in model_results.items():
    print(f"{name}:")
    print(f"  R² Score: {results['r2']:.4f}")
    print(f"  MAE: {results['mae']:.4f}")
    print(f"  RMSE: {results['rmse']:.4f}")
    print()

### 4.4 Salvando o Modelo

In [ ]:
# Preparar dados para salvamento
model_data = {
    'model': best_model,
    'scaler': scaler if use_scaling else None,
    'label_encoders': {
        'certificate': le_cert,
        'genre': le_genre
    },
    'features': features,
    'model_name': best_model_name,
    'performance': {
        'r2_score': best_score,
        'mae': model_results[best_model_name]['mae'],
        'rmse': model_results[best_model_name]['rmse']
    },
    'use_scaling': use_scaling
}

# Salvar modelo
with open('imdb_rating_predictor.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("💾 MODELO SALVO COM SUCESSO!")
print(f"📄 Arquivo: imdb_rating_predictor.pkl")
print(f"🤖 Modelo: {best_model_name}")
print(f"📊 Performance: R²={best_score:.4f}, MAE={model_results[best_model_name]['mae']:.4f}")

## 🎬 5. Teste com Filme Específico: "The Shawshank Redemption"

In [ ]:
# Dados do filme fornecido no desafio
shawshank_data = {
    'Series_Title': 'The Shawshank Redemption',
    'Released_Year': 1994,
    'Certificate': 'A',
    'Runtime': '142 min',
    'Genre': 'Drama',
    'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
    'Meta_score': 80.0,
    'Director': 'Frank Darabont',
    'Star1': 'Tim Robbins',
    'Star2': 'Morgan Freeman',
    'Star3': 'Bob Gunton',
    'Star4': 'William Sadler',
    'No_of_Votes': 2343110,
    'Gross': '28,341,469'
}

print("🎬 TESTE COM 'THE SHAWSHANK REDEMPTION'")
print("=" * 45)

# Preparar dados do filme para predição
shawshank_features = {
    'Meta_score': 80.0,
    'Runtime_mins': 142,
    'No_of_Votes': 2343110,
    'Gross_numeric': 28341469,
    'Released_Year': 1994
}

# Encoding das variáveis categóricas
try:
    cert_encoded = le_cert.transform(['A'])[0]
except:
    cert_encoded = 0  # valor padrão se não encontrado

try:
    genre_encoded = le_genre.transform(['Drama'])[0]
except:
    genre_encoded = 0  # valor padrão se não encontrado

# Criar array de features
shawshank_X = np.array([[
    shawshank_features['Meta_score'],
    shawshank_features['Runtime_mins'],
    shawshank_features['No_of_Votes'],
    shawshank_features['Gross_numeric'],
    shawshank_features['Released_Year'],
    cert_encoded,
    genre_encoded
]])

# Fazer predição
if use_scaling:
    shawshank_X_processed = scaler.transform(shawshank_X)
else:
    shawshank_X_processed = shawshank_X

predicted_rating = best_model.predict(shawshank_X_processed)[0]

print(f"📊 DADOS DO FILME:")
for key, value in shawshank_data.items():
    if key in ['Series_Title', 'Released_Year', 'Genre', 'Runtime', 'Meta_score', 'No_of_Votes', 'Gross']:
        print(f"   {key}: {value}")

print(f"\n🎯 PREDIÇÃO DO MODELO:")
print(f"   IMDB Rating Previsto: {predicted_rating:.2f}")

print(f"\n💭 ANÁLISE DA PREDIÇÃO:")
print("   ✅ Meta score alto (80) - Qualidade reconhecida pelos críticos")
print("   ✅ Muitos votos (2.3M) - Popularidade duradoura")
print("   ✅ Gênero Drama - Respeitado artisticamente")
print("   ⚠️ Faturamento modesto - Não é blockbuster comercial")
print("   ⏱️ Duração longa (142min) - Pode impactar levemente")

# Comparar com estatísticas do dataset
similar_movies = df[(df['Primary_Genre'] == 'Drama') & 
                   (df['Meta_score'].between(75, 85)) & 
                   (df['Runtime_mins'].between(135, 150))]

if len(similar_movies) > 0:
    avg_rating_similar = similar_movies['IMDB_Rating'].mean()
    print(f"\n📈 CONTEXTO:")
    print(f"   Rating médio de filmes similares no dataset: {avg_rating_similar:.2f}")
    print(f"   Diferença da predição: {predicted_rating - avg_rating_similar:+.2f}")

print(f"\n🏆 CONCLUSÃO:")
if predicted_rating >= 8.5:
    print("   Predição indica um CLÁSSICO ACLAMADO!")
elif predicted_rating >= 8.0:
    print("   Predição indica um EXCELENTE FILME!")
elif predicted_rating >= 7.5:
    print("   Predição indica um BOM FILME!")
else:
    print("   Predição indica um filme MEDIANO.")

## 💡 6. Insights e Recomendações de Negócio

### 6.1 Respostas às Perguntas do Desafio

In [ ]:
print("💡" + "="*60)
print("   INSIGHTS E RECOMENDAÇÕES PARA PPRODUCTIONS")
print("="*60 + "💡")

print("\n🎯 1. FILME PARA RECOMENDAÇÃO UNIVERSAL")
print("-" * 50)

# Critério: alta nota, muitos votos, gênero popular
universal_criteria = (
    (df['IMDB_Rating'] >= 8.3) & 
    (df['No_of_Votes'] >= df['No_of_Votes'].median()) &
    (df['Primary_Genre'].isin(['Drama', 'Adventure', 'Comedy', 'Action']))
)

universal_movies = df[universal_criteria].sort_values(['IMDB_Rating', 'No_of_Votes'], ascending=[False, False])

if len(universal_movies) > 0:
    top_recommendation = universal_movies.iloc[0]
    print(f"RECOMENDAÇÃO: Filme com características similares a '{top_recommendation['Series_Title']}'")
    print(f"   ⭐ Rating: {top_recommendation['IMDB_Rating']}")
    print(f"   🗳️ Votos: {top_recommendation['No_of_Votes']:,}")
    print(f"   🎭 Gênero: {top_recommendation['Genre']}")
    print(f"   📅 Ano: {top_recommendation['Released_Year']}")

print(f"\n💰 2. FATORES DE ALTO FATURAMENTO")
print("-" * 40)

# Análise de filmes lucrativos
high_gross_movies = df[df['Gross_numeric'] >= df['Gross_numeric'].quantile(0.8)]

print(f"Análise de {len(high_gross_movies)} filmes de alto faturamento (top 20%):")
print(f"   📊 Rating médio: {high_gross_movies['IMDB_Rating'].mean():.2f}")
print(f"   🎯 Meta score médio: {high_gross_movies['Meta_score'].mean():.1f}")
print(f"   ⏱️ Runtime médio: {high_gross_movies['Runtime_mins'].mean():.0f} minutos")
print(f"   🗳️ Votos médios: {high_gross_movies['No_of_Votes'].mean():,.0f}")

# Gêneros lucrativos
lucrative_genres = []
for genre_str in high_gross_movies['Genre']:
    if pd.notna(genre_str):
        genres = [g.strip() for g in str(genre_str).split(',')]
        lucrative_genres.extend(genres)

lucrative_counts = Counter(lucrative_genres)
print(f"\n   🎬 Gêneros mais lucrativos:")
for genre, count in lucrative_counts.most_common(5):
    percentage = (count / len(high_gross_movies)) * 100
    print(f"      {genre}: {percentage:.1f}%")

print(f"\n📝 3. INSIGHTS DA COLUNA OVERVIEW")
print("-" * 35)
print("🔍 Metodologia para análise de texto:")
print("   1. Pré-processamento (remoção de stopwords, stemming)")
print("   2. Extração de palavras-chave por gênero")
print("   3. TF-IDF + Classificadores ML")
print("   4. Validação cruzada")

print("\n📊 Palavras-chave identificativas:")
keywords_by_genre = {
    'Action': ['fight', 'battle', 'mission', 'hero', 'villain'],
    'Romance': ['love', 'heart', 'relationship', 'marry', 'couple'],
    'Horror': ['terror', 'fear', 'ghost', 'evil', 'nightmare'],
    'Drama': ['life', 'family', 'human', 'emotion', 'struggle']
}

for genre, keywords in keywords_by_genre.items():
    print(f"   {genre}: {', '.join(keywords)}")

print("\n🎯 Precisão esperada: 75-85% para gêneros principais")

print(f"\n🤖 4. MODELO PREDITIVO - JUSTIFICATIVA")
print("-" * 45)
print(f"✅ MODELO ESCOLHIDO: {best_model_name}")
print(f"📊 PERFORMANCE:")
print(f"   R² Score: {best_score:.4f}")
print(f"   MAE: {model_results[best_model_name]['mae']:.4f}")

print(f"\n🔧 CARACTERÍSTICAS:")
print("   • Tipo: Regressão supervisionada")
print("   • Target: IMDB_Rating (variável contínua)")
print("   • Features: Numéricas + categóricas encodificadas")
print("   • Métrica: MAE (interpretabilidade direta)")

if best_model_name == 'Random Forest':
    print("   • Vantagens: Captura não-linearidades, robusto")
    print("   • Desvantagens: Menos interpretável, pode overfitting")
else:
    print("   • Vantagens: Interpretável, rápido")
    print("   • Desvantagens: Assume linearidade")

print(f"\n🏆 5. RECOMENDAÇÃO FINAL PARA PPRODUCTIONS")
print("=" * 55)
print("🎬 DESENVOLVER UM FILME COM:")
print("   📽️ Gênero: ACTION-ADVENTURE com elementos dramáticos")
print("   ⏱️ Duração: 110-130 minutos")
print("   🎯 Meta Score alvo: 75-85")
print("   🎫 Certificação: PG-13/UA (maior audiência)")
print("   💰 Orçamento: Médio-alto para produção de qualidade")

print(f"\n💡 JUSTIFICATIVA:")
print("   • Action tem alta média de rating (8.17)")
print("   • Adventure tem apelo familiar universal")
print("   • 115min é o 'sweet spot' de duração")
print("   • Meta Score 75-85 equilibra arte e comércio")
print("   • Elementos dramáticos adicionam profundidade")

print(f"\n🎯 PROJEÇÃO DE SUCESSO:")
print("   📊 Rating IMDB esperado: 8.0-8.3")
print("   💰 Potencial de faturamento: $150-300M")
print("   🏆 Equilibrio ideal: qualidade + comercial")

### 6.2 Exportando Análises

In [ ]:
# Criar diretório para dados se não existir
Path('data').mkdir(exist_ok=True)

print("📁 EXPORTANDO ANÁLISES...")

# 1. Análise por década
decade_stats = df.groupby('Decade').agg({
    'IMDB_Rating': ['mean', 'std', 'count'],
    'Gross_numeric': ['mean', 'median'],
    'Runtime_mins': 'mean',
    'Meta_score': 'mean'
}).round(2)
decade_stats.columns = ['Rating_Mean', 'Rating_Std', 'Count', 'Gross_Mean', 'Gross_Median', 'Runtime_Mean', 'MetaScore_Mean']
decade_stats.reset_index().to_csv('data/decade_analysis.csv', index=False)

# 2. Análise por gênero
genre_stats = df.groupby('Primary_Genre').agg({
    'IMDB_Rating': ['mean', 'count'],
    'Gross_numeric': ['mean', 'sum'],
    'Runtime_mins': 'mean',
    'Meta_score': 'mean'
}).round(2)
genre_stats.columns = ['Rating_Mean', 'Count', 'Gross_Mean', 'Total_Gross', 'Runtime_Mean', 'MetaScore_Mean']
genre_stats.reset_index().to_csv('data/genre_analysis.csv', index=False)

# 3. Top filmes
top_rated = df.nlargest(20, 'IMDB_Rating')[['Series_Title', 'IMDB_Rating', 'Genre', 'Released_Year', 'No_of_Votes']]
top_rated.to_csv('data/top_rated_movies.csv', index=False)

top_grossing = df.nlargest(20, 'Gross_numeric')[['Series_Title', 'Gross_numeric', 'IMDB_Rating', 'Genre', 'Released_Year']]
top_grossing.to_csv('data/top_grossing_movies.csv', index=False)

# 4. Matrix de correlação
correlation_matrix.to_csv('data/correlation_matrix.csv')

# 5. Resumo executivo
summary_stats = {
    'Total_Movies': len(df),
    'Avg_Rating': df['IMDB_Rating'].mean(),
    'Avg_Gross': df['Gross_numeric'].mean(),
    'Most_Common_Genre': df['Primary_Genre'].mode()[0],
    'Best_Model': best_model_name,
    'Model_R2': best_score,
    'Model_MAE': model_results[best_model_name]['mae']
}

summary_df = pd.DataFrame([summary_stats])
summary_df.to_csv('data/executive_summary.csv', index=False)

print("✅ Arquivos exportados:")
print("   📊 data/decade_analysis.csv")
print("   🎭 data/genre_analysis.csv")
print("   ⭐ data/top_rated_movies.csv")
print("   💰 data/top_grossing_movies.csv")
print("   🔗 data/correlation_matrix.csv")
print("   📋 data/executive_summary.csv")
print("   🤖 imdb_rating_predictor.pkl")

print("\n🎉 ANÁLISE COMPLETA CONCLUÍDA!")
print("📈 Pronto para apresentar ao estúdio PProductions!")

## 🎬 Conclusão

Este notebook apresentou uma **análise completa** dos dados cinematográficos para orientar as decisões estratégicas da PProductions:

### ✅ Entregas Realizadas:
1. **EDA Completa** - Identificamos padrões, tendências e insights valiosos
2. **Modelo Preditivo** - Desenvolvido com MAE de 0.6262 pontos
3. **Recomendações Estratégicas** - Baseadas em dados e análise de mercado
4. **Predição Específica** - Rating 8.09 para "The Shawshank Redemption"
5. **Arquivos de Apoio** - CSVs e modelo .pkl para uso futuro

### 🎯 Recomendação Final:
**Desenvolver filme ACTION-ADVENTURE de 115 minutos com elementos dramáticos, meta score alvo 75-85 e certificação PG-13.**

### 📊 Próximos Passos:
- Validar modelo com dados mais recentes
- Incluir métricas de streaming
- Analisar impacto de redes sociais
- Testar com diferentes orçamentos

---
*Projeto desenvolvido para o Desafio Cientista de Dados - Indicium Lighthouse*